In [13]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from helita.sim import rh15d 
from helita.vis import rh15d_vis

from scipy.integrate import quadrature
from scipy.integrate import cumtrapz

from pathlib import Path

rhRepoPath = Path("/Users/elka127/Documents/code/rh")

# Atoms for CaII

```
# Nmetal
   10

# Metals
#  model file              ACTIVE/PASSIVE  INITIAL_SOLUTION   population file


# ../../Atoms/CaII.atom            PASSIVE    ZERO_RADIATION
  ../../Atoms/H_6.atom             PASSIVE    ZERO_RADIATION
  ../../Atoms/MgII-IRIS.atom       PASSIVE    ZERO_RADIATION
  ../../Atoms/CaII_PRD.atom        ACTIVE     ZERO_RADIATION
  ../../Atoms/Si.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/Al.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/Fe.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/He.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/N.atom               PASSIVE    LTE_POPULATIONS
  ../../Atoms/Na.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/S.atom               PASSIVE    LTE_POPULATIONS
```

In [10]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
wave = data.ray.wavelength
indices = np.arange(len(wave))[(wave > 392.8) & (wave < 394.0)]

wave.sel(wavelength=500, method="nearest")
index500 = np.argmin(np.abs(wave.data - 500))

with open(rhRepoPath / "rh15d/run/ray.input", "w") as f:
    f.write("1.00\n")
    output = str(len(indices) + 1)
    for ind in indices:
        output += f" {ind}"
    output += f" {index500}\n"
    f.write(output)

--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_aux.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_indata.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_ray.hdf5 file.


# Intencity plot for Ca II

In [26]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")

--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_aux.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_indata.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_ray.hdf5 file.


In [31]:
data.ray.intensity.plot()
plt.axis([392.8, 394.0, 0, 2e-9])

(392.8, 394.0, 0.0, 2e-09)

# Calculating optical depths

In [35]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
print(data.ray.wavelength_selected)
print("-")
print(data.ray.wavelength_selected.data)

# first column
height = data.atmos.height_scale[0, 0].dropna('height')  
# index of 500 nm
index500 = np.argmin(np.abs(data.ray.wavelength_selected.data - 500))  

tau500 = cumtrapz(data.ray.chi[0, 0, :, index500].dropna('height'), x=-height)
# ensure tau500 and height have same size
tau500 = np.concatenate([[1e-20], tau500])  

 

--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_aux.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_indata.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_ray.hdf5 file.
<xarray.DataArray 'wavelength_selected' (wavelength_selected: 44)>
array([278.180447, 278.182993, 278.185539, 278.188085, 278.190631, 278.193177,
       278.195723, 278.198269, 278.200815, 278.203361, 278.205907, 278.208453,
       278.210999, 278.213545, 278.218637, 278.221183, 278.223729, 278.226275,
       278.228821, 278.231367, 278.233913, 278.236459, 278.239005, 278.241551,
       278.244097, 278.246643, 278.249189, 278.251735, 278.254281, 278.256827,
       278.259373, 278.261919, 278.264465, 278.267011, 278.269557, 278.272103,
       278.274649, 278.277195, 278.279741, 278.282287, 278.284833, 278.287379,
       278.289925, 278.562348])
Coordinates:
  * wavelength_selected  (wavelength_selected) float64 278.2 278.2 ... 278.6
Attributes:


In [29]:
fig, ax = plt.subplots()
ax.plot(height / 1e6, tau500)  # height in Mm
ax.set_xlabel('H (Mm)')
ax.set_ylabel(r'$\tau$$_{500}$')
ax.set_yscale('log')
ax.axhline(y=1,linestyle='dashed',color='black')
ax.axvline(x=0.5,linestyle='dashed',color='black')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Plot the departure coefficients for the levels of Ca II



In [50]:
# plt.figure()
# for i in range(5):
#     plt.plot(data.atmos.height_scale[0,0]/1e6, 
#              data.atom_CA.populations[i, 0, 0]/data.atom_CA.populations_LTE[i, 0, 0],
#              label='Level %i' % (i + 1))
# plt.legend(loc="upper left")
# plt.figure()
# for i in range(5):
#     plt.plot(data.atmos.height_scale[0,0]/1e6, 
#              data.atom_CA.populations[i, 0, 0]/data.atom_CA.populations_LTE[i, 0, 0],
#              label='Level %i' % (i + 1))
fig, ax = plt.subplots()
ax.plot(tau500,data.atom_CA.populations[1, 0, 0]/data.atom_CA.populations_LTE[1, 0, 0], label='Level %1' % (1)) 
plt.legend(loc="upper left")
plt.xlabel("tau500")
plt.ylim(0.05, 2)
plt.ylabel("Departure coefficients")
plt.ylim(0.05, 2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

# от тау 500

In [ ]:
plt.figure()
for i in range(5):
    plt.plot(data.atmos.height_scale[0,0]/1e6, 
             data.atom_CA.populations[i, 0, 0]/data.atom_CA.populations_LTE[i, 0, 0],
             label='Level %i' % (i + 1))
plt.legend(loc="upper left")
plt.xlabel("Height (Mm)")
plt.ylabel("Departure coefficients")
plt.ylim(0.05, 2)

# Source function

In [34]:
rh15d_vis.SourceFunction(data);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='wavelength', max=43), Checkbox(value=True, description='…